# Loading & Preparing Data

In [2]:
from os import listdir;
import numpy as np;
from keras.utils import np_utils;
from keras.callbacks import ModelCheckpoint;
import sys;
from keras.models import Sequential;
from keras.layers import Dense, LSTM, Dropout;

seqLength = 100;
path = "Stories/";
files = [f for f in listdir(path) if ("DS_Store" not in f)];
rawData = "";
startingData = [];
uniqueWords = [];
for f in files:
    text = open(path + f).read();
    
    text = text.replace("\t", " ");
    text = text.replace('–', " - ");
    text = text.replace('—', " - ");
    text = text.replace('‘', " ' "); 
    text = text.replace('…', " . ");
    text = text.replace('ç', "c");
    text = text.replace('é', "e");
    text = text.lower();
    
    rawData += text;
    startingData.append(text[0:seqLength]);
    
chars = sorted(list(set(rawData)));
numChars = len(chars);
numData = len(rawData);

print("%d stories has been loaded"%len(files));
print("Number of different characters: %d"%numChars);
print("Different characters: " + str(chars));
print("Number of characters in the text: %d"%numData);

charInt = dict((c, i) for i,c in enumerate(chars));
intChar = dict((i, c) for i,c in enumerate(chars));

print("Converting input data to sequences...");
dataX = [];
dataY = [];
for i in range(numData - seqLength):
    seqIn = rawData[i:i+seqLength];
    seqOut = rawData[i+seqLength];
    
    dataX.append([charInt[c] for c in seqIn]);
    dataY.append(charInt[seqOut]);
    
numPatterns = len(dataX);
print("All data has been converted...");
print("Number of patterns: %d"%numPatterns);
print("Reshaping the data");
X = np.reshape(dataX, (numPatterns, seqLength, 1));
X = X/float(numChars);
Y = np_utils.to_categorical(dataY);

135 stories has been loaded
Number of different characters: 51
Different characters: ['\n', ' ', '!', '"', '#', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Number of characters in the text: 3044178
Converting input data to sequences...
All data has been converted...
Number of patterns: 3044078
Reshaping the data


# Training Deep Tingle

In [ ]:
model = Sequential();
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True));
model.add(Dropout(0.2));
model.add(LSTM(256));
model.add(Dropout(0.2));
model.add(Dense(Y.shape[1], activation='softmax'));
model.load_weights("AliceWeights/alltingle-weights-improvement-45-1.2487.hdf5");
model.compile(loss='categorical_crossentropy', optimizer='adam');
# define the checkpoint
filepath="new-tingleweights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5";
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min');
callbacks_list = [checkpoint];
# fit the model
model.fit(X, Y, nb_epoch=100, batch_size=128, callbacks=callbacks_list);

# Generate New Story

In [21]:
numOfCharacters = 1000;

model = Sequential();
model.add(LSTM(256, batch_input_shape=(1, X.shape[1], X.shape[2]), return_sequences=True, stateful=True))
model.add(Dropout(0.2))
model.add(LSTM(256, stateful=True));
model.add(Dropout(0.2));
model.add(Dense(Y.shape[1], activation='softmax'))
model.load_weights("AliceWeights/alltingle-weights-improvement-45-1.2487.hdf5");
model.compile(loss="categorical_crossentropy", optimizer="adam");
startText = startingData[np.random.randint(0, len(startingData))];
pattern = [charInt[c] for c in startText];
result = startText + "";
print("Starting Text: \"" + startText + "\"\n");

result = " ";
i = 0;
while i < numOfCharacters or result != " ":
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(numChars)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = intChar[index]
    seq_in = [intChar[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    i+= 1;
    
print("\n\nThe end\n");

Starting Text: "i don't really think of myself as a criminal, but if i were hard pressed to answer with a definitive"

 cock and the sensation of the sensation of prostate orgasm begins to stretch out of my mouth. i can feel my thick body and then start to stroke my hands and literele and then start to beat off my hands and then start to stretch out and start to stretch out and start to stretch out and start to stretch out of my mouth. i can feel the sensation of pleasure is almost in a series of pleasure and then suddenly the sensation of the sensation of the sensation of his shaft is all the sensation of pleasure and then suddenly the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of pleasure. i can feel the sensation of powerful press street and then starts to stretch out of me and then stands up and down across the beast and then start to stroke my hands and lisser my hands and knees. i stand up and start to stand up and then start to

# Character by Character New Story

In [20]:
from keras.models import Sequential;
from keras.layers import Dense, LSTM, Dropout;
import sys
import numpy

numOfCharacters = 1000;

model = Sequential();
model.add(LSTM(256, batch_input_shape=(1, 1, 1), return_sequences=True, stateful=True))
model.add(Dropout(0.2))
model.add(LSTM(256, stateful=True));
model.add(Dropout(0.2));
model.add(Dense(Y.shape[1], activation='softmax'))
model.load_weights("AliceWeights/alltingle-weights-improvement-45-1.2487.hdf5");
model.compile(loss="categorical_crossentropy", optimizer="adam");
startText = startingData[np.random.randint(0, len(startingData))];
pattern = [charInt[c] for c in startText];
print("Starting Text: \"" + startText + "\"\n");

result = " ";
i = 0;
while i < numOfCharacters or result != " ":
    currentInput = [pattern[0]];
    x = numpy.reshape(currentInput, (1, len(currentInput), 1))
    x = x / float(numChars)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = intChar[index]
    seq_in = [intChar[value] for value in pattern]
    pattern = pattern[1:len(pattern)];
    if len(pattern) == 0:
        pattern.append(index);
        sys.stdout.write(result);
    else:
        sys.stdout.write(intChar[currentInput[0]]);
    i+= 1;
    
print("\n\nThe end\n");

Starting Text: "public transportation can be a real pain in the ass, especially when you're used to getting around w"

public transportation can be a real pain in the ass, especially when you're used to getting around ith the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the short that i can feel a strange strange and the sensation of prostate orgasm begins to stretch out of my mouth. i can feel the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the sensation of the s